# Average Toxicity Score
Three regression models **RandomForestRegressor, GradientBoostingRegressor and KernelRidge** are trained based on the data from the previous competition of 4 years ago, where the task was **multi-label classification** of offensive and neutral comments.

**Toxicity score** is estimated based on the **adjusted sum of all values in all categories**. Final prediction is the average prediction of best regressors selected using validation data. 

Text preprocessing is done by **TfidfVectorizer** with the maximum number of **features limited to 20,000** after applying text cleaning function to remove special characters.

In [ ]:
import re
from bs4 import BeautifulSoup
import os
import random
import joblib
import numpy as np
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, cross_val_score

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.kernel_ridge import KernelRidge

import matplotlib.pyplot as plt

In [ ]:
TEST_DATA_PATH = '../input/jigsaw-toxic-severity-rating/comments_to_score.csv'
VALID_DATA_PATH = '../input/jigsaw-toxic-severity-rating/validation_data.csv'
TRAIN_DATA_PATH = '../input/d/julian3833/jigsaw-toxic-comment-classification-challenge/train.csv'

In [ ]:
SEED = 10

## Functions

In [ ]:
def set_seed(seed=42):
    """Utility function to use for reproducibility.
    :param seed: Random seed
    :return: None
    """
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)


def set_display():
    """Function sets display options for charts and pd.DataFrames.
    """
    # Plots display settings
    plt.style.use('fivethirtyeight')
    plt.rcParams['figure.figsize'] = 12, 8
    plt.rcParams.update({'font.size': 14})
    # DataFrame display settings
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    pd.options.display.float_format = '{:.4f}'.format
    
    
def text_cleaning(text: str) -> str:
    """Function cleans text removing special characters,
    extra spaces, embedded URL links, HTML tags and emojis.
    Code source: https://www.kaggle.com/manabendrarout/pytorch-roberta-ranking-baseline-jrstc-infer
    :param text: Original text
    :return: Preprocessed text
    """
    template = re.compile(r'https?://\S+|www\.\S+')  # website links
    text = template.sub(r'', text)

    soup = BeautifulSoup(text, 'lxml')  # HTML tags
    only_text = soup.get_text()
    text = only_text

    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    text = re.sub(r"[^a-zA-Z\d]", " ", text)  # special characters
    text = re.sub(' +', ' ', text)  # extra spaces
    text = text.strip()  # spaces at the beginning and at the end of string

    return text

## Data processing

In [ ]:
set_seed(SEED)
set_display()

By the rules of this competition we can use public data sets on Kaggle. However for some reason we cannot submit notebooks that directly use the data from the previous competition of 4 years ago where text samples were labeled with 6 classes. In this version we use an exact copy of this competition data set created and published by [Julian Peller](https://www.kaggle.com/julian3833) [here](https://www.kaggle.com/julian3833/jigsaw-toxic-comment-classification-challenge).

In [ ]:
# Extract classified text samples and clean the texts.
data_train = pd.read_csv(TRAIN_DATA_PATH)
data_train['comment_text'] = data_train['comment_text'].apply(text_cleaning)
data_train.head()

In [ ]:
data_train.describe()

In [ ]:
categories = data_train.loc[:, 'toxic':'identity_hate'].sum()
plt.title('Category Frequency')
plt.bar(categories.index, categories.values)
plt.show()

In the previous competition the task was to perform multi-class classification. Text sample could be labeled with one or several categories or not labeled with any. Non-toxic comments represent the majority of text samples, while toxic comments are a minority class and extremely toxic comments are more rare than plain toxic.

In this competition we have to score texts based on the level of toxicity. To get a toxicity score from the previous data we can use two approaches:
- Simply sum up all values in each row of the DataFrame. The toxicity score will vary between 0 and 6. However some unequally toxic samples could have the same score.
- Adjust the values in the DataFrame according to extremety of the category (for example, "toxic" and "severe toxic" should have different score) and then sum up per row values.

In [ ]:
scores = data_train.loc[:, 'toxic':'identity_hate'].sum(axis=1).value_counts()
plt.bar(scores.index, scores.values)
plt.title('Scores Distribution: Simple Sum')
plt.show()

In the original DataFrame each category contains binary values (0 or 1). We will change the original values using multiplicative factors observing the following (disputable) common sense rules:
- Normal non-offensive text samples would have a score of 0.
- "toxic" category with a score of 1 would be used as a benchmark to score other offensive categories.
- Samples marked as "severe_toxic" would have higher toxicity score than those marked as "toxic".
- Obscene language along would have slightly lower score than "toxic" samples.
- Insults would be scored in between "toxic" and "severe_toxic" closer to the upper bound.
- Samples containing threats would have the highest toxicity score.
- Identity hate would be scored marginally lower than threats.
- If a sample is marked as belonging to several offensive classes total score would be calculated as a sum of values in all individual categories.

In [ ]:
# Multiplication factors for categories.
cat_mtpl = {'toxic': 1, 'severe_toxic': 1.75, 'obscene': 0.95,
            'threat': 2, 'insult': 1.6, 'identity_hate': 1.95}

for category in cat_mtpl:
    data_train[category] = data_train[category] * cat_mtpl[category]

data_train['score'] = data_train.loc[:, 'toxic':'identity_hate'].sum(axis=1)

In [ ]:
plt.hist(data_train['score'])
plt.title('Scores Distribution: Adjusted Sum')
plt.show()

Data set is imbalanced: number of neutral text samples is much larger than the number of toxic samples. We limit number of samples with 0 toxicity score to 1/5 of the total number of texts labeled as toxic.

In [ ]:
n_samples_toxic = len(data_train[data_train['score'] != 0])
n_samples_normal = len(data_train) - n_samples_toxic

idx_to_drop = data_train[data_train['score'] == 0].index[n_samples_toxic//5:]
data_train = data_train.drop(idx_to_drop)

print(f'Reduced number of neutral text samples from {n_samples_normal} to {n_samples_toxic//5}.')
print(f'Total number of training samples: {len(data_train)}')

In [ ]:
print(f'Mean toxicity score: {data_train["score"].mean()}\n'
      f'Standard deviation: {data_train["score"].std()}')

## Define the models

In [ ]:
# Candidate models
kridge = make_pipeline(
    TfidfVectorizer(decode_error='ignore', stop_words='english', 
                    max_features=20_000, ngram_range=(1, 2), min_df=2),
    KernelRidge(alpha=1.0)
)

kridge2 = make_pipeline(
    TfidfVectorizer(decode_error='ignore', stop_words='english', 
                    max_features=20_000, ngram_range=(1, 2), min_df=2),
    KernelRidge(alpha=0.9)
)

randforest = make_pipeline(
    TfidfVectorizer(decode_error='ignore', stop_words='english', 
                    max_features=20_000, ngram_range=(1, 2), min_df=2),
    RandomForestRegressor(n_jobs=-1)
)

randforest2 = make_pipeline(
    TfidfVectorizer(decode_error='ignore', stop_words='english', 
                    max_features=20_000, ngram_range=(1, 2), min_df=2),
    RandomForestRegressor(n_estimators=150, n_jobs=-1)
)

In [ ]:
models = [
    ('KernelRidge', kridge),
    ('KernelRidge2', kridge2),
    ('RandomForest', randforest),
    ('RandomForest2', randforest2)
]

## Train and validate the models

In [ ]:
# New data for validation: text pairs.
data_valid = pd.read_csv(VALID_DATA_PATH)

# Clean the texts
data_valid['less_toxic'] = data_valid['less_toxic'].apply(text_cleaning)
data_valid['more_toxic'] = data_valid['more_toxic'].apply(text_cleaning)

data_valid.head()

In [ ]:
# Train each model on all available samples from previous competition.
for name, model in models:
    print('-' * 50)
    model.fit(data_train['comment_text'], data_train['score'])
    print(f'{name} model completed training.')

    # Estimate toxicity score for text pairs.
    data_valid[f'less_toxic_score_{name}'] = model.predict(data_valid['less_toxic'])
    data_valid[f'more_toxic_score_{name}'] = model.predict(data_valid['more_toxic'])
    print(f'{name} model completed prediction.')

    # Compare scores for all text pairs.
    data_valid[f'result_{name}'] = \
        data_valid[f'more_toxic_score_{name}'] > data_valid[f'less_toxic_score_{name}']

    # Ratio of correctly scored text pairs.
    print('Correct predictions:', data_valid[f'result_{name}'].sum() / len(data_valid))

    joblib.dump(model, f'{name}.joblib')

In [ ]:
# Check the accuracy of averaged scores from the best models.
columns = [f'less_toxic_score_{name}' for name, _ in models]
data_valid['less_toxic_score'] = data_valid[columns].mean(axis=1)

columns = [f'more_toxic_score_{name}' for name, _ in models]
data_valid['more_toxic_score'] = data_valid[columns].mean(axis=1)

data_valid[f'result'] = data_valid[f'more_toxic_score'] > data_valid[f'less_toxic_score']
print('Correct averaged predictions:', data_valid[f'result'].sum() / len(data_valid))

## Prediction on the test set

In [ ]:
# New data for text scoring.
data_test = pd.read_csv(TEST_DATA_PATH)
data_test['text'] = data_test['text'].apply(text_cleaning)
data_test.head()

In [ ]:
# Get prediction from the best models.
for name, model in models:
    data_test[f'score_{name}'] = model.predict(data_test['text'])
    print(f'{name} model completed prediction.')

In [ ]:
# Average the result.
columns = [f'score_{name}' for name, _ in models]
data_test['score'] = data_test[columns].mean(axis=1)

In [ ]:
data_test[['comment_id', 'score']].to_csv('submission.csv', index=False)
data_test[['comment_id', 'score']].head()

<p style="text-align:center;font-size:20px;color:blue"> If you've made it to the end of this notebook and found something useful,</p>
<p style="text-align:center;font-size:20px;color:blue"> please, upvote! 😉</p>